In [ ]:
import numpy as np
from tvsclib.canonical_form import CanonicalForm
from tvsclib.mixed_system import MixedSystem
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.system_identification_svd import SystemIdentificationSVD

import timeit

# Notebook with basic explaination

## Create a System

First we will create a system to explore the capabilities of the library.
For this we use the `SystemIdentificationSVD`
The output and input dims have to be given.
We can also give an $\epsilon$. This determeines if smaller singular values of the hankel matrices should be ignored.

The library also allows to print the system.
This gives basic informations about the structure. 

In [ ]:
dims_in =  [2, 1, 2, 1, 5, 2,10, 3, 2, 1, 3, 2, 4, 2, 5,20,30,10,10,10,15]
dims_out = [1, 2, 1, 2, 5, 2, 7, 3, 2, 1, 5, 7, 2, 1, 2,20,30,10,10,10,15]
matrix = np.random.rand(sum(dims_out), sum(dims_in))
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-10)
system = MixedSystem(S)


print(system)

## Extract causal and anticausal part

As we can see the (mixed)system contains a casual system and an anticausal system.
These can be extracted form the mixed system.

In [ ]:
system_causal = system.causal_system
system_anticausal = system.anticausal_system


## Calculate the output of the system

Now we can apply the syste to a input vector $u$.
This can be done using the funtion `compute`.

This returns the output vector $y$ and a vector $x$ containing the states

In [ ]:
#create a input vector
u = np.random.rand(sum(dims_in),1)


x_s, y_s = system.compute(u)

#calcaulate reference
y = matrix@u

np.max(abs(y-y_s))

## Get properties

It is also possible to get the properties of the system.
We can determine if the function is minimal using the functions `is_minimal`
The functions `is_observable` and `is_reachable` are also available.
These can be given an optional tolerance for the determination of the rank.

In [ ]:
system.is_minimal()

This is also possible for the causal and anticausal systems:

In [ ]:
system.is_minimal()

With the functions `ìs_input_normal`,`ìs_input_normal` and `ìs_balanced` we can check if a strict system has a normal form

In [ ]:
system_causal.is_input_normal()

In [ ]:
system_anticausal.is_balanced(tolerance=1e-12)

## Convert system back to matrix

It is also possible to convert the system back to a matrix.
This can be done with the function `to_matrix`.
The function usually uses the Block matrix representation

$$
\begin{bmatrix}
   D_1 &       &     &  \cdots   \\
   C_2B_1 & D_2&     &  \cdots   \\
   C_3A_2B_1 & C_3B_2& D_3 & \cdots \\
   \vdots    & \vdots& \vdots & \ddots
\end{bmatrix}
$$
Thsi formaultion is usefull for large matrices.
With the option `use_formula=True` the matrix is calculated using the formula
$$T = D + C(I − Z@A)^{−1}ZB$$
This is mainly interesting for theoretical purposes.

The same can be done for the anticausla system.
If `to_matrix` is called on a mixed system the matrix of both the causal and the anticausal are calcaulated and added.

In [ ]:
matrix_rec = system_causal.to_matrix(use_formula=True)+ system_anticausal.to_matrix(use_formula=True)
np.max(matrix-matrix_rec)

In [ ]:
matrix_constr = system_causal.to_matrix(use_formula=False)+ system_anticausal.to_matrix(use_formula=False)
np.max(matrix-matrix_constr)

Compare the speed of the different implementations

In [ ]:
timeit.timeit(lambda:system_causal.to_matrix(use_formula=False), number=10)

In [ ]:
timeit.timeit(lambda:system_causal.to_matrix(use_formula=True), number=10)